<a href="https://colab.research.google.com/github/seonae0223/machine_learning/blob/main/08_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xgboost==1.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 MB 6.5 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    Uninstalling xgboost-2.0.3:
      Successfully uninstalled xgboost-2.0.3


In [2]:
import xgboost as xgb

print(xgb.__version__)

/usr/local/lib/python3.10/dist-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


1.5.0


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df['target'] = cancer.target
data_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [5]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(
    data_df.drop("target", axis=1),
    data_df['target'],
    random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, # X_train 데이터 세트를 기준으로 한번 더 데이터를 자름
    y_train,
    random_state=42
)

# DMatrix 변환
- 넘파이 배열, 판다스 데이터 프레임을 DMatrix로 변환

In [ ]:
!pip install pandas==1.5

In [7]:
dtr = xgb.DMatrix(data=X_train, label=y_train)
dval = xgb.DMatrix(data=X_valid, label=y_valid)
dtest = xgb.DMatrix(data=X_test, label=y_test)

#XGB 하이퍼 파라미터 설정

In [10]:
params = {
    "max_depth": 3,
    'eat': 0.05,
    'objective':'binary:logistic',
    'eval_metric': 'logloss', # 0과 1 사이의 loss일 때, 조금 더 강조된 loss를 확인할 수 있음

}

#최대 400회 훈련 -> 조기 중단 기능(Early Stopping)을 사용할 것이기 때문에 400번을 다 훈련할 필요X
num_rounds = 400

## 훈련

In [13]:
# 평가, 훈련 세트를 함께 넣는 리스트
eval_list = [
    (dtr, 'train'), # 훈련하는 데이터
    (dval, 'eval') # 훈련한 것에 대해 평가하는 데이터
]

xgb_model = xgb.train(
    params=params, #앞서 지정한 하이퍼 파라미터
    dtrain=dtr, # 훈련 데이터 세트 따로 지정
    num_boost_round=num_rounds, # 최대 학습 횟수
    early_stopping_rounds=50, # 성능 개선이 50 라운드 이내에 이루어지지 않으면 학습 중단
    evals=eval_list # 평가 리스트 지정

)

[00:46:51] WARNING: ../src/learner.cc:576: 
Parameters: { "eat" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-logloss:0.47223	eval-logloss:0.48805
[1]	train-logloss:0.34045	eval-logloss:0.37900
[2]	train-logloss:0.25744	eval-logloss:0.31620
[3]	train-logloss:0.19984	eval-logloss:0.27947
[4]	train-logloss:0.15985	eval-logloss:0.24494
[5]	train-logloss:0.13100	eval-logloss:0.23093
[6]	train-logloss:0.11008	eval-logloss:0.21262
[7]	train-logloss:0.09038	eval-logloss:0.20279
[8]	train-logloss:0.07607	eval-logloss:0.19069
[9]	train-logloss:0.06517	eval-logloss:0.18585
[10]	train-logloss:0.05697	eval-logloss:0.17404
[11]	train-logloss:0.05017	eval-logloss:0.16949
[12]	train-logloss:0.04466	eval-logloss:0.16308
[13]	train-logloss:0.04038	eval-